In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache
from langchain.callbacks import get_openai_callback

set_llm_cache(SQLiteCache("cache.db"))

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

examples = [
    {
        "movie": "Shrek",
        "answer": """
        Here is what I know:
        Director: Andrew Adamson, Vicky Jenson
        Cast: Mike Myers (Shrek), Eddie Murphy (Donkey), Cameron Diaz (Princess Fiona), John Lithgow (Lord Farquaad)
        Budget: $60 million
        Box office revenue: $487.9 million
        Genre: Animation, Comedy, Adventure, Fantasy
        Brief Synopsis: A grumpy but kind-hearted ogre named Shrek embarks on a journey to rescue Princess Fiona, encountering a talkative donkey and uncovering unexpected secrets along the way.
        """,
    },
    {
        "movie": "Top Gun",
        "answer": """
        I know this:
        Director: Tony Scott
        Cast: Tom Cruise (Maverick), Kelly McGillis (Charlie), Val Kilmer (Iceman), Anthony Edwards (Goose)
        Budget: $15 million
        Box office revenue: $357.3 million
        Genre: Action, Drama
        Brief Synopsis: A talented but reckless Navy pilot, Pete "Maverick" Mitchell, trains at the elite Top Gun Naval Fighter Weapons School, facing rivalries, romance, and personal loss while striving to prove himself.
        """,
    },
    {
        "movie": "Spider-Man",
        "answer": """
        I know this:
        Director: Sam Raimi
        Cast: Tobey Maguire (Peter Parker/Spider-Man), Kirsten Dunst (Mary Jane Watson), Willem Dafoe (Norman Osborn/Green Goblin), James Franco (Harry Osborn)
        Budget: $139 million
        Box office revenue: $825 million
        Genre: Action, Superhero, Adventure
        Brief Synopsis: After being bitten by a genetically modified spider, high school student Peter Parker gains superpowers and learns the responsibility of being Spider-Man while facing his first major villain, the Green Goblin.
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {movie}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {movie}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"movie": "Jurassic world"})


# with get_openai_callback() as usage:
#     a = chat.predict("What is the recipe for soju")
#     b = chat.predict("What is the recipe for bread")
#     print(a, "\n")
#     print(b, "\n")
#     print(usage)

In [21]:
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

# LLM 초기화
llm = ChatOpenAI(temperature=0.1)

# ✅ 메모리 클래스 변경: 대화를 유지하는 ConversationBufferMemory 사용
memory = ConversationBufferMemory(
    return_messages=True
)

# ✅ Few-Shot 예제 추가 (세 개의 이모티콘으로 답하도록)
examples = [
    {"movie": "Top Gun", "answer": "🛩️👨‍✈️🔥"},
    {"movie": "The Godfather", "answer": "👨‍👨‍👦🔫🍝"},
    {"movie": "Titanic", "answer": "🚢💔🎶"},
    {"movie": "Jurassic Park", "answer": "🦖🏃🌿"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [("human", "What do you know about {movie}?"), ("ai", "{answer}")]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# ✅ 시스템 프롬프트 및 메모리 적용
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert. You always reply with exactly three emojis."),
        few_shot_prompt,  # 예제 추가
        MessagesPlaceholder(variable_name="history"),  # 메모리 활용
        ("human", "{movie}"),
    ]
)

# ✅ LCEL 체인 구현: 메모리 활용하도록 설정
def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

# ✅ 체인 실행 함수
def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )
    print(result.content)

/var/folders/q5/m8wmkz_d1c11pbrck3p2lm_m0000gn/T/ipykernel_94214/95251428.py:11: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [22]:
invoke_chain("Top Gun")

🛩️👨‍✈️🔥


In [23]:
invoke_chain("The Godfather")

👨‍👨‍👦🔫🍝


In [24]:
invoke_chain("What was the first movie I asked about?")

🛩️👨‍✈️🔥
